In [ ]:
!pip install nibabel
!pip install pyradiomics
!pip install pynrrd
!pip install SimpleITK

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
import nrrd
from scipy import ndimage
import matplotlib.pyplot as plt
import os
import glob
import radiomics
from radiomics import featureextractor
import logging
import SimpleITK as sitk

In [ ]:
# Mask conversion.

def convert_masks(filepath):
    # Load the segmentation mask.
    seg = nib.load(filepath)
    seg_data = seg.get_fdata()

    # Define binary masks.
    ne_mask = np.where(seg_data == 1, 1, 0)  # non-enhancing tumor core
    en_mask = np.where(seg_data == 3, 1, 0)  # enhancing tumor

    # Combine binary masks to define whole tumor.
    binary_mask = np.logical_or(ne_mask, en_mask)  # binary mask

    # Convert boolean mask to uint8.
    binary_mask = binary_mask.astype(np.uint8)

    # Define file name.
    basename = os.path.basename(filepath).replace('seg.nii.gz', '')
    binary_mask_file = os.path.join(os.path.dirname(filepath), basename + 'seg_bin.nii.gz')

    # Save new binary masks as NIfTI files.
    binary_mask = nib.Nifti1Image(binary_mask, seg.affine)
    nib.save(binary_mask, binary_mask_file)

    print(f"Saved binary mask for {basename}.")

# Find all the files in directory.
seg_files = glob.glob("/content/drive/MyDrive/BraTS-MEN/Training/**/*seg.nii.gz", recursive=True)

# Apply mask conversion to each file.
for file in seg_files:
    convert_masks(file)

In [ ]:
import os

def list_patient_files(base_path, patient_id):
    """List all files for a given patient, returns an empty list if the directory does not exist."""
    patient_path = os.path.join(base_path, str(patient_id))
    if os.path.exists(patient_path):
        return [f for f in os.listdir(patient_path) if os.path.isfile(os.path.join(patient_path, f))]
    else:
        return []

def has_required_modalities(patient_files):
    """Check if all required modalities are present."""
    required_modalities = ['t1c.nii.gz', 't1n.nii.gz', 't2w.nii.gz', 't2f.nii.gz']
    for modality in required_modalities:
        if not any(modality in file for file in patient_files):
            return False
    return True

def has_required_masks(patient_files):
    """Check if all required modalities are present."""
    required_masks = ['seg_bin.nii.gz']
    for mask in required_masks:
        if not any(mask in file for file in patient_files):
            return False
    return True

base_path = '/content/drive/MyDrive/BraTS-MEN/Training'

In [ ]:
p = '/content/drive/MyDrive/BraTS-MEN/Training'
patients = os.listdir(p)
print(len(patients))

In [ ]:
patients

In [ ]:
radiomics.logger.setLevel(logging.ERROR)

extractor = featureextractor.RadiomicsFeatureExtractor('/content/drive/MyDrive/BraTS-CPTAC/params.yaml')

In [ ]:
# Set the file paths
sample_path = '/content/drive/MyDrive/BraTS-MEN/Training/BraTS-MEN-00004-000/BraTS-MEN-00004-000-t1c.nii.gz'
sample_mask_path = '/content/drive/MyDrive/BraTS-MEN/Training/BraTS-MEN-00004-000/BraTS-MEN-00004-000-seg_bin.nii.gz'

# Read the image and mask using SimpleITK
sample_image = sitk.ReadImage(sample_path)
sample_mask = sitk.ReadImage(sample_mask_path)

# Execute the feature extraction
sample_features = extractor.execute(sample_image, sample_mask)

# Create a list of feature names, excluding diagnostic features
feature_names = [k for k in sample_features.keys() if not k.startswith('diagnostics')]

In [ ]:
# Function to extract features or return zeros if an error occurs
def extract_features_or_zero(extractor, image, mask, index, feature_names):
    try:
        features = extractor.execute(image, mask)
        features = {k: v for k, v in features.items() if not k.startswith('diagnostics')}
    except ValueError as e:
        print(f'Error extracting features: {e}')
        # Dynamically create a dictionary with zeros for all feature names
        features = {feature_name: np.nan for feature_name in feature_names}

    return pd.DataFrame([features], index=index)

In [ ]:
# Lists to store the feature DataFrames for each patient and MRI type
T1, T1c, T2, FLAIR = ([] for _ in range(4))

for patient in patients:
    index = [patient]

    # Construct the file paths for each image and mask type
    T1_p = p + '/' + patient + '/' + patient + '-t1n.nii.gz'
    T1c_p = p + '/' + patient + '/' + patient + '-t1c.nii.gz'
    T2_p = p + '/' + patient + '/' + patient + '-t2w.nii.gz'
    FLAIR_p = p + '/' + patient + '/' + patient + '-t2f.nii.gz'

    m_p = p + '/' + patient + '/' + patient + '-seg_bin.nii.gz'

    # Read the images and masks using SimpleITK
    T1_image = sitk.ReadImage(T1_p)
    T1c_image = sitk.ReadImage(T1c_p)
    T2_image = sitk.ReadImage(T2_p)
    FLAIR_image = sitk.ReadImage(FLAIR_p)

    mask = sitk.ReadImage(m_p)

    print('Extracting features of: ' + str(patient))

    # Use the function for each image-mask pair
    T1.append(extract_features_or_zero(extractor, T1_image, mask, index, feature_names))
    T1c.append(extract_features_or_zero(extractor, T1c_image, mask, index, feature_names))
    T2.append(extract_features_or_zero(extractor, T2_image, mask, index, feature_names))
    FLAIR.append(extract_features_or_zero(extractor, FLAIR_image, mask, index, feature_names))

    print(f'Finished extracting features for patient: {patient}'+'\n')

In [ ]:
# Concatenate the DataFrames for each mask and MRI type.

T1_df = pd.concat(T1)
T1_df = T1_df.add_prefix('T1_')

T1c_df = pd.concat(T1c)
T1c_df = T1c_df.add_prefix('T1c_')

T2_df = pd.concat(T2)
T2_df = T2_df.add_prefix('T2_')

FLAIR_df = pd.concat(FLAIR)
FLAIR_df = FLAIR_df.add_prefix('FLAIR_')

combined = pd.concat([T1_df, T1c_df, T2_df, FLAIR_df], axis=1)
combined.to_csv('/content/drive/MyDrive/BraTS-MEN/radiomic_features_train.csv')